Скачаем данные

In [104]:
import json
import pandas as pd
import gzip
import random
from string import punctuation
import re
import gensim
import nltk
from nltk.collocations import *
import pandas as pd
import logging

In [5]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Digital_Music_5.json.gz')

In [140]:
def lowercase(t):
    return str(t).lower()

In [141]:
df['reviewText'] = df['reviewText'].apply(lowercase)

In [142]:
df

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,3,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"this is awesome to listen to, a must-have for ...",Slayer Rules!,1370217600,NaN
1,5.0,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5.0,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,it was great to hear the old stuff again and i...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4.0,3,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5.0,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,what can i say? this is casting crowns!!!this ...,"This is a good, blessing filled",1465689600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
169776,5.0,NaN,True,"08 19, 2018",A1SR2T84IXOMAQ,B01HJ91MTW,{'Format:': ' MP3 Music'},Diane B.,casting crowns songs are all wonderful!!,Great song!!,1534636800,NaN
169777,5.0,NaN,True,"03 18, 2018",A2SR3DWJR1PYR6,B01HJ91MTW,{'Format:': ' MP3 Music'},jan c.,just reminds you that you are never alone.,Five Stars,1521331200,NaN
169778,5.0,NaN,True,"03 7, 2018",A24V7X30NIMOIY,B01HJ91MTW,{'Format:': ' MP3 Music'},Lori Y,"good product, good service.",Five Stars,1520380800,NaN
169779,5.0,NaN,True,"02 12, 2018",A1LW10GYP2EYM1,B01HJ91MTW,{'Format:': ' MP3 Music'},N F,i love every single song this group sings. the...,Absolutely beautiful...,1518393600,NaN


Ниже предлагаю способы, воспользуюсь word2vec



| Способ  | Необходимые данные   | Плюсы    | Минусы |
| ------- | -------------------- | -------- | -------- | 
| Выбирать текст в кавычках   | Только корпус         | + можно сделать, потому что мы смотрим на названия песен <br /> + не нужны дополнительные данные данные   | - скорее всего люди не особо используют кавычки в отзывах...    |
| Векторизовать корпус c помощью word2vec   | Только корпус            |  + можно найти все слова, близкие к song/album/music, и быстро вытащить все близкие слова    | - не очень понятно, хватит ли объёма корпуса, чтобы всё выделить    |
|  n-граммы с ключевыми словами  | только корпус                |  + частое явление для длинных отзывов    | - не все отзывы длинные <br /> - непонятно, какой брать n и где ставить границу в случае название песен и альбомов    |
| достать названия с помощью Natasha | только корпус | + большинство наименований тут - названия песен, они могут доставаться | - но это неточно |

In [143]:
reviews = '\n'.join(str(a) for a in list(df['reviewText']))
with open('reviews.txt', 'w', encoding='utf-8') as f:
    f.write(reviews)
data = gensim.models.word2vec.LineSentence('reviews.txt')
%time model_rev = gensim.models.Word2Vec(data, vector_size=500, window=5, min_count=5, epochs=10)

CPU times: total: 3min
Wall time: 1min 1s


In [144]:
print(len(model_rev.wv))

36215


In [163]:
def get_more_words(list_start):
    final_namings = []
    for name in my_namings:
        for (word, cos) in model_rev.wv.most_similar(name):
            if cos >= 0.45:
                if word[-1] in punctuation:
                    final_namings.append(word[:-1])
                else:
                    final_namings.append(word)
    final_namings = list(set(final_namings))
    return final_namings

In [164]:
my_namings = ['song', 'album', 'track']
final = get_more_words(my_namings)

In [165]:
final

["album's",
 'record',
 'song..',
 'song',
 'cd',
 'tune',
 'album',
 'effort',
 'ep',
 'track',
 'verse',
 'disc',
 'tracks',
 'cut',
 'one',
 'it',
 'song!']

Уберем лишние варианты ручками

In [167]:
final.remove('song!')
final.remove('it')
final.remove('one')
final.remove('song..')
final.remove("album's")

In [169]:
final

['record',
 'song',
 'cd',
 'tune',
 'album',
 'effort',
 'ep',
 'track',
 'verse',
 'disc',
 'tracks',
 'cut']

In [170]:
texts = df['reviewText'].tolist()

In [171]:
texts_list = []
for a in texts:
    new_a = str(a).split()
    texts_list.append(new_a)

In [172]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(texts_list)

In [173]:
all_clcs_likelihood = finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [174]:
all_clcs_raw_freq = finder2.score_ngrams(bigram_measures.raw_freq)

In [175]:
all_clcs_dice = finder2.score_ngrams(bigram_measures.dice)

Мы посчитали частотности всех биграмм, а надо только с нашими словами, сделаем функцию, которая достает только нужные биграммы

In [176]:
stopwords = ['of', 'stop', 'the', 'to', 'and', 'a', 'in', 'it', 'is', 'I', 'that', 'had', 'on', 'for', 'were', 'was', 'this']

In [177]:
def filter_bigr(all_clcs):
    appr = []
    for (bi, ratio) in all_clcs: #проверяем, что одно из слов из нашего списка
        if bi[0] in final or bi[1] in final: #
            if bi[0] not in stopwords and bi[1] not in stopwords:
                appr.append([bi, ratio])
    return appr

In [178]:
filtered_likelihood = filter_bigr(all_clcs_likelihood)
filtered_raw = filter_bigr(all_clcs_raw_freq)
filtered_dice = filter_bigr(all_clcs_dice)

In [179]:
filtered_likelihood[:100]

[[('great', 'song'), 23755.63419651684],
 [('good', 'song'), 9092.290696535518],
 [('song', 'from'), 4250.824957351624],
 [('title', 'track'), 4051.4914671586957],
 [('song', 'by'), 3544.56061281201],
 [('i', 'song'), 3223.621830263607],
 [('an', 'album'), 2987.198429637527],
 [('whole', 'album'), 2041.3473190306072],
 [('beautiful', 'song'), 1930.849706182636],
 [('every', 'song'), 1853.7559378334743],
 [('awesome', 'song'), 1746.697752283045],
 [('tracks', 'are'), 1729.8801028031444],
 [('entire', 'album'), 1664.627223938844],
 [('favorite', 'song'), 1614.178043687118],
 [('song', 'because'), 1590.767265976114],
 [('debut', 'album'), 1545.5443516672378],
 [('catchy', 'tune'), 1515.0143962963332],
 [('song', 'song'), 1460.2561894396067],
 [('song', 'has'), 1446.6324756270606],
 [('fun', 'song'), 1422.6623547231384],
 [('bonus', 'tracks'), 1366.5185438509993],
 [('nice', 'song'), 1162.9705392690614],
 [('best', 'song'), 1095.9090166894161],
 [('new', 'album'), 1090.4603258086456],
 [('

In [181]:
filtered_raw[:100]

[[('great', 'song'), 0.0012698737145779247],
 [('good', 'song'), 0.0005706521146134223],
 [('song', 'from'), 0.00035715766510540514],
 [('song', 'i'), 0.00027150529226189915],
 [('song', 'by'), 0.0002702323270604033],
 [('song', 'with'), 0.00021531297122443977],
 [('song', 'has'), 0.0001869440324482467],
 [('an', 'album'), 0.0001452998851421684],
 [('favorite', 'song'), 0.00011856761591075568],
 [('song', 'about'), 0.00011656724202269079],
 [('song', 'because'), 0.00011602168550776399],
 [('beautiful', 'song'), 0.00011547612899283721],
 [('every', 'song'), 0.0001120209377316342],
 [('best', 'song'), 0.00011020241601521156],
 [('song', 'when'), 0.00010838389429878894],
 [('awesome', 'song'), 0.00010711092909729309],
 [('great', 'album'), 0.00010638352041072404],
 [('song', 'but'), 0.00010256462480623651],
 [('song', 'as'), 9.492683359726145e-05],
 [('one', 'song'), 9.474498142561919e-05],
 [('nice', 'song'), 9.165349450770071e-05],
 [('song', 'so'), 8.838015541813998e-05],
 [('fun', 'so

In [182]:
filtered_dice[:100]

[[('title', 'track'), 0.1257108650104759],
 [('great', 'song'), 0.12273701972088445],
 [('catchy', 'tune'), 0.07539178314273613],
 [('good', 'song'), 0.0671294563113027],
 [('awkward', 'cut'), 0.06405693950177936],
 [('bonus', 'tracks'), 0.06244343891402715],
 [('cut', 'off.'), 0.05964585274930102],
 [('sophomore', 'effort'), 0.05442176870748299],
 [('record', 'company'), 0.05291005291005291],
 [('closing', 'cut'), 0.052478134110787174],
 [('an', 'album'), 0.04772001075044047],
 [('record', 'companies'), 0.04483837330552659],
 [('bonus', 'disc'), 0.04311377245508982],
 [('song', 'from'), 0.04297451943590473],
 [('solid', 'effort'), 0.040549378678875085],
 [('record', 'label'), 0.03857957036387549],
 [('opening', 'track'), 0.03712055507372073],
 [('disc', '1'), 0.03696718219539796],
 [('song', 'by'), 0.035667994815419324],
 [('blu-ray', 'disc'), 0.03423423423423423],
 [('whole', 'album'), 0.0333494441759304],
 [('title', 'cut'), 0.02986312733305682],
 [('disc', '2'), 0.02935545628589661

На этом моменте мне начало казаться, что digital музыка была не самым лучшим выбором, но что теперь поделать. Мне понравились результаты третьего способа (меры Дайса) - там интересные коллокации вроде 2nd cd, catchy song и так далее, но первые две (лайклихуд и простая частотность) тоже неплохо справились. Выберем коллокаций для track, song, cd, album и ep по мере Дайса

In [185]:
bigramms = []
for (bi, r) in filtered_dice[:500]:
    bigramms.append(' '.join(bi))

In [188]:
print('track')
print('---')
for bi in bigramms:   
    if 'track' in bi:
        print(bi)
print('---')
print('song')
for bi in bigramms:
    if 'song' in bi:
        print(bi)
print('---')
print('cd')
for bi in bigramms:
    if 'cd' in bi:
        print(bi)
print('---')
print('album')
for bi in bigramms:
    if 'album' in bi:
        print(bi)
print('---')
print('ep')
for bi in bigramms:
    if 'ep' in bi:
        print(bi)

track
---
title track
bonus tracks
opening track
tracks are
standout tracks
tracks include
every track
these tracks
few tracks
bonus track
track listing:
favorite tracks
favorite track
tracks here
other tracks
two tracks
each track
closing track
instrumental tracks
tracks from
final track
track listing
track from
11 tracks
great. track
extra tracks
first track
best tracks
tracks here,
tracks 1.
16 tracks
track set
sound track
tracks here.
last track
ten tracks
four tracks
tracks are:
several tracks
live tracks
best track
favourite track
five tracks
tracks like
weak track
tracks such
additional tracks
18 tracks
8 track
some tracks
tracks 1
three tracks
individual tracks
tracks including
single track
track 3
tracks include:
12 tracks
track here
those tracks
8 tracks
standout track
digital tracks
track off
track with
track by
solid track
track wasn't
tracks included
2 tracks
eight tracks
next track
strongest tracks
second track
3 tracks
bad track
14 tracks
lost track
tracks quickly
track 